# SVM и его ядра
__Суммарное количество баллов: 10__

В этом задании вам не нужно будет реализовать SVM, а научиться его применять.

Примените метод SVM (например, из библиотеки sklearn) для датасета blobs2.

Визуализируйте результат (разбиение плоскости и опорные вектора) при разных вариантах ядер (линейное; полиномиальное степеней 2,3,5; RBF).

In [ ]:
from sklearn.svm import SVC 
from sklearn.datasets import make_blobs
import seaborn as sns
import numpy as np

X_blobs2, y_blobs2  = make_blobs(n_samples=1000, random_state=4, center_box=(-2, 2),cluster_std=0.4, centers=2)
print(X_blobs2, y_blobs2)

In [ ]:
linear_svc = SVC(kernel='linear')
poly_2_svc = SVC(kernel='poly', degree=2)
poly_3_svc = SVC(kernel='poly', degree=3)
poly_5_svc = SVC(kernel='poly', degree=5)
rbf_svc = SVC(kernel='rbf')

[svc.fit(X_blobs2, y_blobs2) for svc in [linear_svc, poly_2_svc, poly_3_svc, poly_5_svc, rbf_svc]]

sns.scatterplot(data=X_blobs2)


Протестируйте SVM на датасете spam. Для этой задачи используйте данные x_spam_train и y_spam_train:

1. Выполните загрузку и предобработку файлов x_spam_train и y_spam_train.
2. Разбейте x_spam_train и y_spam_train на x_train, y_train, x_test и y_test для оценки точности работы алгоритма.
3. Посчитайте метрики precision, recall, accuracy для модели SVM. Если необходимо, попробуйте разные наборы параметров для получения лучшего результата.
4. Сравните время работы SVM и Random Forest при близкой точности.

Ответьте на следующие вопросы:
1. Какой нужен препроцессинг данных для моделей?
2. Какая модель делает предсказания лучше? Предположите, почему.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

from sklearn.ensemble import RandomForestClassifier

Обучите модель на всех данных из x_spam_train и y_spam_train.
Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = test["Id"]
submission["Expected"] = #YOUR CODE
submission.to_csv('submission.csv', index=False)